![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/CLASSIFICATION_AGE_GROUP.ipynb)

## **CLASSIFICATION AGE GROUP**

## **Colab Setup**

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 5.2.2
Spark NLP_JSL Version : 5.2.1


## **bert_sequence_classifier_age_group**

In [ ]:
text_list = ["""I'm only 25 and was prescribed lisnopril for high blood pressure. I also suffer from frequent headaches and migraines. Now the lisnopril actually did help with my headaches but the side effects are just too much. (Difficulty speaking, not myself, weakness) I made the decision to stop taking them. It's been 7 days and I literally can barely talk without my voice squeaking and sounding retarded. Im like on the verge of losing my job fuxk these poison pills like wtf do they give out to people.""",
"""Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; loose stools/ pale stools, elevated liver enzymes, low iron.  5 months and still no answers from drs.""",
"""Patient was brought to the hospital due to lethargy, feeding difficulties, and difficulty breathing. Upon examination, patient was found to have a high-grade fever and increased respiratory rate. Lab results showed a high white blood cell count and chest X-ray revealed pneumonia. The patient was immediately started on intravenous antibiotics and oxygen therapy. Due to the severity of the infection, the treatment was continued for several days until the patient's fever subsided and oxygen saturation levels improved. The patient was also given nebulized bronchodilators to ease breathing difficulties. After a week, patient was discharged from the hospital with a prescription for oral antibiotics, which was continued at home for a few more days. The patient's parents were counseled on proper hygiene measures such as handwashing and avoiding close contact with sick individuals to prevent future infections.""",
"""A child patient presents to the emergency room with complaints of fever, cough, and difficulty breathing. Upon examination, the child is found to have an elevated heart rate and low oxygen saturation levels. The child is diagnosed with pneumonia and is started on oral antibiotics immediately. Additionally, the child is given a nebulizer treatment with albuterol, a bronchodilator, to help with breathing. The family is told to monitor the child's condition closely and bring them back to the hospital if the symptoms worsen or do not improve within a few days.""",
"""Patient presents with symptoms of severe anxiety and depression. They report difficulty sleeping, loss of appetite, and a lack of interest in activities they previously enjoyed. Upon further discussion, it is discovered that the patient has been regularly abusing prescription drugs, specifically benzodiazepines and opioids. The patient is referred to a specialist to address their substance abuse and provided with therapy and medication for their mental health issues.""",
"""I have been taking Alendronic Acid for 2 years after breaking my femur and was diagnosed with Osteoporosis, I know drugs have side effects but I was not told about coming off them and the effects, getting some problems with it and would love a herbal or natural alternative I am 65 soon. My recent scan was good but I would prefer not to take this drug. How do I wean myself off this drug or can I just quit. help please thank you."""
]

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

sequenceClassifier = MedicalBertForSequenceClassification.pretrained("bert_sequence_classifier_age_group", "en", "clinical/models")\
    .setInputCols(["document",'token'])\
    .setOutputCol("prediction")

clf_Pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    sequenceClassifier])


pipeline_model = clf_Pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

bert_sequence_classifier_age_group download started this may take some time.
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = pipeline_model.transform(df)

In [ ]:
x = result.select("document.result", "prediction.result")
df = x.toDF('text', 'class')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+---------+
|                                                                                                text|    class|
+----------------------------------------------------------------------------------------------------+---------+
|[I'm only 25 and was prescribed lisnopril for high blood pressure. I also suffer from frequent he...|  [Adult]|
|[Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; l...|  [Child]|
|[Patient was brought to the hospital due to lethargy, feeding difficulties, and difficulty breath...|[Unknown]|
|[A child patient presents to the emergency room with complaints of fever, cough, and difficulty b...|  [Child]|
|[Patient presents with symptoms of severe anxiety and depression. They report difficulty sleeping...|[Unknown]|
|[I have been taking Alendronic Acid for 2 years after breaking my femur and was diagnosed with 

## **genericclassifier_age_group_sbiobert_cased_mli**

In [ ]:
text_list = ["""Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; loose stools/ pale stools, elevated liver enzymes, low iron.  5 months and still no answers from drs.""",
"""Patient was brought to the hospital due to lethargy, feeding difficulties, and difficulty breathing. Upon examination, patient was found to have a high-grade fever and increased respiratory rate. Lab results showed a high white blood cell count and chest X-ray revealed pneumonia. The patient was immediately started on intravenous antibiotics and oxygen therapy. Due to the severity of the infection, the treatment was continued for several days until the patient's fever subsided and oxygen saturation levels improved. The patient was also given nebulized bronchodilators to ease breathing difficulties. After a week, patient was discharged from the hospital with a prescription for oral antibiotics, which was continued at home for a few more days. The patient's parents were counseled on proper hygiene measures such as handwashing and avoiding close contact with sick individuals to prevent future infections.""",
"""A child patient presents to the emergency room with complaints of fever, cough, and difficulty breathing. Upon examination, the child is found to have an elevated heart rate and low oxygen saturation levels. The child is diagnosed with pneumonia and is started on oral antibiotics immediately. Additionally, the child is given a nebulizer treatment with albuterol, a bronchodilator, to help with breathing. The family is told to monitor the child's condition closely and bring them back to the hospital if the symptoms worsen or do not improve within a few days.""",
"""Patient presents with symptoms of severe anxiety and depression. They report difficulty sleeping, loss of appetite, and a lack of interest in activities they previously enjoyed. Upon further discussion, it is discovered that the patient has been regularly abusing prescription drugs, specifically benzodiazepines and opioids. The patient is referred to a specialist to address their substance abuse and provided with therapy and medication for their mental health issues.""",
"""I have been taking Alendronic Acid for 2 years after breaking my femur and was diagnosed with Osteoporosis, I know drugs have side effects but I was not told about coming off them and the effects, getting some problems with it and would love a herbal or natural alternative I am 65 soon. My recent scan was good but I would prefer not to take this drug. How do I wean myself off this drug or can I just quit. help please thank you."""
]

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", 'en','clinical/models')\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

features_asm = FeaturesAssembler()\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("features")

generic_classifier = GenericClassifierModel.pretrained("genericclassifier_age_group_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")

pipeline = Pipeline(stages=[
    document_assembler,
    sentence_embeddings,
    features_asm,
    generic_classifier
])


pipeline_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
genericclassifier_age_group_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = pipeline_model.transform(df)

In [ ]:
x = result.select("document.result", "prediction.result")
df = x.toDF('text', 'class')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+---------+
|                                                                                                text|    class|
+----------------------------------------------------------------------------------------------------+---------+
|[Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; l...|  [Child]|
|[Patient was brought to the hospital due to lethargy, feeding difficulties, and difficulty breath...|[Unknown]|
|[A child patient presents to the emergency room with complaints of fever, cough, and difficulty b...|  [Child]|
|[Patient presents with symptoms of severe anxiety and depression. They report difficulty sleeping...|[Unknown]|
|[I have been taking Alendronic Acid for 2 years after breaking my femur and was diagnosed with Os...|  [Adult]|
+-----------------------------------------------------------------------------------------------

## **few_shot_classifier_age_group_sbiobert_cased_mli**

In [ ]:
text_list = ["""Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; loose stools/ pale stools, elevated liver enzymes, low iron.  5 months and still no answers from drs.""",
"""Patient was brought to the hospital due to lethargy, feeding difficulties, and difficulty breathing. Upon examination, patient was found to have a high-grade fever and increased respiratory rate. Lab results showed a high white blood cell count and chest X-ray revealed pneumonia. The patient was immediately started on intravenous antibiotics and oxygen therapy. Due to the severity of the infection, the treatment was continued for several days until the patient's fever subsided and oxygen saturation levels improved. The patient was also given nebulized bronchodilators to ease breathing difficulties. After a week, patient was discharged from the hospital with a prescription for oral antibiotics, which was continued at home for a few more days. The patient's parents were counseled on proper hygiene measures such as handwashing and avoiding close contact with sick individuals to prevent future infections.""",
"""A child patient presents to the emergency room with complaints of fever, cough, and difficulty breathing. Upon examination, the child is found to have an elevated heart rate and low oxygen saturation levels. The child is diagnosed with pneumonia and is started on oral antibiotics immediately. Additionally, the child is given a nebulizer treatment with albuterol, a bronchodilator, to help with breathing. The family is told to monitor the child's condition closely and bring them back to the hospital if the symptoms worsen or do not improve within a few days.""",
"""Patient presents with symptoms of severe anxiety and depression. They report difficulty sleeping, loss of appetite, and a lack of interest in activities they previously enjoyed. Upon further discussion, it is discovered that the patient has been regularly abusing prescription drugs, specifically benzodiazepines and opioids. The patient is referred to a specialist to address their substance abuse and provided with therapy and medication for their mental health issues."""
]

In [ ]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

bert_sent = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["document"])\
    .setOutputCol("sentence_embeddings")

few_shot_classifier = FewShotClassifierModel.pretrained("few_shot_classifier_age_group_sbiobert_cased_mli", "en", "clinical/models")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("prediction")

clf_Pipeline = Pipeline(stages=[
    document_assembler,
    bert_sent,
    few_shot_classifier
])

pipeline_model = clf_Pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]
few_shot_classifier_age_group_sbiobert_cased_mli download started this may take some time.
[OK!]


In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text" : text_list}))

result = pipeline_model.transform(df)

In [ ]:
x = result.select("document.result", "prediction.result")
df = x.toDF('text', 'class')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+---------+
|                                                                                                text|    class|
+----------------------------------------------------------------------------------------------------+---------+
|[Hi, wondering if anyone has had a similar situation. My 1 year old daughter has the following; l...|  [Child]|
|[Patient was brought to the hospital due to lethargy, feeding difficulties, and difficulty breath...|[Unknown]|
|[A child patient presents to the emergency room with complaints of fever, cough, and difficulty b...|  [Child]|
|[Patient presents with symptoms of severe anxiety and depression. They report difficulty sleeping...|[Unknown]|
+----------------------------------------------------------------------------------------------------+---------+



## **genericclassifier_age_e5**

In [4]:
sample_texts = [
"""The patient presents with conditions often associated with the stresses and lifestyle of early career and possibly higher education stages, including sleep irregularities and repetitive stress injuries. There's a notable emphasis on preventative care, with discussions around lifestyle choices that can impact long-term health, such as smoking cessation, regular exercise, and balanced nutrition. The patient is also counseled on mental health, particularly in managing stress and anxiety that may arise from personal and professional responsibilities and ambitions at this stage of life.""",
"""The patient, in the infancy stage, is experiencing typical developmental milestones such as teething and learning to crawl. Regular check-ups focus on monitoring growth patterns, ensuring proper nutrition, especially with the transition from milk to solid foods, and addressing common infant concerns like colic and sleep patterns. Immunizations are up-to-date, following the recommended schedule for this critical early life stage.""",
"""The toddler patient shows normal signs of curiosity and motor skill development for their age. Concerns include potential delays in speech development and the challenges of transitioning from crib to bed. Nutritional guidance is provided, focusing on balanced diets and managing the picky eating phase common in toddlers. Safety in the home environment is also discussed, given the patient's increased mobility and exploratory behavior.""",
"""The senior patient presents with age-related issues such as reduced hearing and vision, arthritis, and memory lapses. Emphasis is on managing chronic conditions, maintaining social engagement, and adapting lifestyle to changing physical abilities. Discussions include medication management, dietary adjustments to suit older digestion, and the importance of regular, low-impact exercise.""",
"""The late teenage patient is dealing with final growth spurts, the stress of impending adulthood, and decisions about higher education or career paths. Health discussions include maintaining a balanced diet, the importance of regular sleep patterns, and managing academic and social pressures. Mental health support is considered crucial at this stage, with a focus on building resilience and coping mechanisms.""",
"""The patient, faces adjustments to a new lifestyle with changes in daily routines and social interactions. Health concerns include managing the transition from an active work life to more leisure time, which may impact physical and mental health. Preventative health measures are emphasized, along with the importance of staying mentally and physically active and engaged in the community."""
]

In [5]:
document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = E5Embeddings.pretrained("e5_large", "en")\
    .setInputCols(["document"])\
    .setOutputCol("embeddings")\
    .setMaxSentenceLength(512)

features_asm = FeaturesAssembler()\
    .setInputCols(["embeddings"])\
    .setOutputCol("features")

generic_classifier = GenericClassifierModel.pretrained("genericclassifier_age_e5", "en", "clinical/models")\
    .setInputCols(["features"])\
    .setOutputCol("prediction")

pipeline = Pipeline(stages=[
    document_assembler,
    embeddings,
    features_asm,
    generic_classifier
])

e5_large download started this may take some time.
Approximate size to download 762 MB
[OK!]
genericclassifier_age_e5 download started this may take some time.
[OK!]


In [6]:
pipeline_model = pipeline.fit(spark.createDataFrame([[""]]).toDF("text"))

In [7]:
df = spark.createDataFrame(pd.DataFrame({"text" : sample_texts}))

result = pipeline_model.transform(df)

In [8]:
x = result.select("document.result", "prediction.result")
df = x.toDF('text', 'class')
df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------+
|                                                                                                text|           class|
+----------------------------------------------------------------------------------------------------+----------------+
|[The patient presents with conditions often associated with the stresses and lifestyle of early c...|         [Adult]|
|[The patient, in the infancy stage, is experiencing typical developmental milestones such as teet...|[Child and Teen]|
|[The toddler patient shows normal signs of curiosity and motor skill development for their age. C...|[Child and Teen]|
|[The senior patient presents with age-related issues such as reduced hearing and vision, arthriti...|     [Old Adult]|
|[The late teenage patient is dealing with final growth spurts, the stress of impending adulthood,...|[Child and Teen]|
|[The patient, faces adjustments to a ne